In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:3


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y
    

In [6]:
loss_thresh = 0.005
label = "1D_FODE_rowdy"

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [7]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)

                      
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
       
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
      
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  x_coll = torch.from_numpy(colloc_pts(N_f,123)).float().to(device)
  f_hat = torch.zeros(x_coll.shape[0],1).to(device)

  for i in range(max_iter):
    
    train_step(i)

    loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val =8.0
rowdy_terms = 2

for reps in range(max_reps):  
  print(label) 
  train_loss = []
  test_mse_loss = []
  test_re_loss = []   
  alpha_val = []
  omega_val = []

  torch.manual_seed(reps*36)
  N_f = 10000 #Total number of collocation points

  layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
  PINN = Sequentialmodel(layers,n_val,rowdy_terms)


  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())

  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                            max_iter = 10, 
                            max_eval = 15, 
                            tolerance_grad = 1e-6, 
                            tolerance_change = 1e-6, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')



  train_model(max_iter,reps)


  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  alpha_full.append(alpha_val)
  omega_full.append(omega_val)


  print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,"omega": omega_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

1D_FODE_rowdy
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.768845 Test MSE 5220.596547879134 Test RE 0.9998002260533937
1 Train Loss 47.768456 Test MSE 5220.482907142386 Test RE 0.9997893442842742
2 Train Loss 47.768326 Test MSE 5220.224729839918 Test RE 0.9997646218484422
3 Train Loss 47.710194 Test MSE 5167.751736326571 Test 

94 Train Loss 0.00681278 Test MSE 0.09827661630497116 Test RE 0.004337887275809401
95 Train Loss 0.006591435 Test MSE 0.1356136487557375 Test RE 0.005095711890664989
96 Train Loss 0.006361981 Test MSE 0.12137844602176098 Test RE 0.0048208536909298974
97 Train Loss 0.006285482 Test MSE 0.0720870439005142 Test RE 0.0037151963637064083
98 Train Loss 0.0062803477 Test MSE 0.0690009523502315 Test RE 0.0036348015713701105
99 Train Loss 0.0062803477 Test MSE 0.0690009523502315 Test RE 0.0036348015713701105
Training time: 70.23
Training time: 70.23
1D_FODE_rowdy
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=5

86 Train Loss 0.0036849594 Test MSE 0.0796252285874429 Test RE 0.003904617628877303
87 Train Loss 0.0036455337 Test MSE 0.07083970311281544 Test RE 0.0036829136027132564
88 Train Loss 0.0035323144 Test MSE 0.045773090271117534 Test RE 0.0029604537564997782
89 Train Loss 0.0033703179 Test MSE 0.031742567235333675 Test RE 0.0024653250510829356
90 Train Loss 0.0032933443 Test MSE 0.041253670227172475 Test RE 0.0028105056388956435
91 Train Loss 0.0032645513 Test MSE 0.042487441429250865 Test RE 0.002852222845448379
92 Train Loss 0.0032455744 Test MSE 0.03903321185460508 Test RE 0.0027338224717953293
93 Train Loss 0.0032133423 Test MSE 0.032672827871446306 Test RE 0.0025011890982055396
94 Train Loss 0.003150333 Test MSE 0.03922173842604975 Test RE 0.0027404165659121846
95 Train Loss 0.003031833 Test MSE 0.02982378135672937 Test RE 0.0023896512182107845
96 Train Loss 0.0029407812 Test MSE 0.042467112129674106 Test RE 0.002851540401079266
97 Train Loss 0.0028038672 Test MSE 0.0993228671660439

80 Train Loss 1.5392646 Test MSE 160.02750682840426 Test RE 0.1750453120434716
81 Train Loss 1.5002371 Test MSE 136.61273506101566 Test RE 0.1617330429773379
82 Train Loss 1.4150976 Test MSE 122.7234274321095 Test RE 0.153291086550862
83 Train Loss 1.2532039 Test MSE 109.53100441655815 Test RE 0.14481771803275562
84 Train Loss 1.1913095 Test MSE 99.35220711333517 Test RE 0.13792466074453835
85 Train Loss 1.1107763 Test MSE 84.5686659832421 Test RE 0.12725003329505155
86 Train Loss 1.0823014 Test MSE 81.37482351833495 Test RE 0.12482402846669383
87 Train Loss 1.0576624 Test MSE 82.94803634306257 Test RE 0.12602485874161592
88 Train Loss 1.0262613 Test MSE 90.79885014004057 Test RE 0.13185400901812344
89 Train Loss 0.9890606 Test MSE 98.6241956801348 Test RE 0.13741840450799478
90 Train Loss 0.97098464 Test MSE 95.18833421315307 Test RE 0.1350035000715833
91 Train Loss 0.95126206 Test MSE 89.50477785597654 Test RE 0.13091104034221981
92 Train Loss 0.91846174 Test MSE 86.80882915076705 Te

73 Train Loss 0.0036992757 Test MSE 0.06775201763961948 Test RE 0.0036017559405153677
74 Train Loss 0.0036548642 Test MSE 0.06154679767668017 Test RE 0.003432858443079713
75 Train Loss 0.003606281 Test MSE 0.045113790517095353 Test RE 0.0029390557488785957
76 Train Loss 0.0034700644 Test MSE 0.027649858827025305 Test RE 0.0023009099587530404
77 Train Loss 0.0033087377 Test MSE 0.04136134260706184 Test RE 0.002814170968917827
78 Train Loss 0.0031373685 Test MSE 0.03606197318054644 Test RE 0.002627712867305388
79 Train Loss 0.0029860928 Test MSE 0.031165970066867242 Test RE 0.0024428313748789967
80 Train Loss 0.002763576 Test MSE 0.006587404697821466 Test RE 0.001123079079250479
81 Train Loss 0.0025239324 Test MSE 0.013485998614430306 Test RE 0.001606921716648135
82 Train Loss 0.0024186773 Test MSE 0.020832058896081987 Test RE 0.001997189444376921
83 Train Loss 0.002389711 Test MSE 0.02067069165663377 Test RE 0.001989439190715432
84 Train Loss 0.002342626 Test MSE 0.019490706714762596 Te

...

57 Train Loss 0.0015305344 Test MSE 0.012695760919925268 Test RE 0.0015591307333883263
58 Train Loss 0.001488349 Test MSE 0.022106127276163238 Test RE 0.002057356238133287
59 Train Loss 0.0014574736 Test MSE 0.012479742539519322 Test RE 0.0015458095199352288
60 Train Loss 0.0014271166 Test MSE 0.003675782809692951 Test RE 0.0008389345605838936
61 Train Loss 0.0014131673 Test MSE 0.0036581015295781183 Test RE 0.0008369144034168714
62 Train Loss 0.0014102001 Test MSE 0.003985065738687 Test RE 0.0008735160913633638
63 Train Loss 0.0014083314 Test MSE 0.004094010632163467 Test RE 0.0008853758011264092
64 Train Loss 0.0014080742 Test MSE 0.004104775305815994 Test RE 0.0008865390278064388
65 Train Loss 0.0014066682 Test MSE 0.004036358252436008 Test RE 0.000879119710810845
66 Train Loss 0.0014061476 Test MSE 0.003979464661844252 Test RE 0.0008729020047397115
67 Train Loss 0.0014054743 Test MSE 0.0038779157546707777 Test RE 0.0008616925729404862
68 Train Loss 0.0014054743 Test MSE 0.003877915

48 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
49 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
50 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
51 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
52 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
53 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
54 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
55 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
56 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
57 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
58 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
59 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015
60 Train Loss 47.769028 Test MSE 5221.510300863163 Test RE 0.999887718973015

44 Train Loss 0.044472314 Test MSE 6.257381909226535 Test RE 0.03461381720167681
45 Train Loss 0.03756629 Test MSE 6.340841158644173 Test RE 0.034843887405392765
46 Train Loss 0.031897917 Test MSE 7.5869054129025955 Test RE 0.03811408511773105
47 Train Loss 0.028029533 Test MSE 4.5492852616111135 Test RE 0.02951377415659407
48 Train Loss 0.0258028 Test MSE 2.890506410077403 Test RE 0.023525568827701474
49 Train Loss 0.02422102 Test MSE 2.9832565179016775 Test RE 0.023900030972376415
50 Train Loss 0.0223454 Test MSE 3.1101756133418688 Test RE 0.024403134892757368
51 Train Loss 0.021040883 Test MSE 2.496625161680457 Test RE 0.021864010952963784
52 Train Loss 0.019329445 Test MSE 2.1628860431749986 Test RE 0.02035026100869495
53 Train Loss 0.01687193 Test MSE 1.2309180988081476 Test RE 0.015352098329419693
54 Train Loss 0.014289323 Test MSE 1.0513369211541441 Test RE 0.014188096147357838
55 Train Loss 0.012994511 Test MSE 0.546850851762112 Test RE 0.010232638017986899
56 Train Loss 0.0124

35 Train Loss 0.48499677 Test MSE 123.48761475637124 Test RE 0.1537676105262987
36 Train Loss 0.40205735 Test MSE 114.75568059134099 Test RE 0.14823141733267076
37 Train Loss 0.33818722 Test MSE 112.83095051687297 Test RE 0.1469830611730978
38 Train Loss 0.25215483 Test MSE 98.49279630539262 Test RE 0.13732683108295177
39 Train Loss 0.22478573 Test MSE 96.66151613736152 Test RE 0.13604417961319432
40 Train Loss 0.20536625 Test MSE 94.23003465583979 Test RE 0.13432221354907262
41 Train Loss 0.18392608 Test MSE 84.76229831459922 Test RE 0.127395628784754
42 Train Loss 0.17228144 Test MSE 79.92933879087937 Test RE 0.12371041814718986
43 Train Loss 0.16653372 Test MSE 76.83424696302313 Test RE 0.12129156069221528
44 Train Loss 0.15718101 Test MSE 72.00767263159763 Test RE 0.11742012863369672
45 Train Loss 0.1491108 Test MSE 71.05867905696073 Test RE 0.1166438188201364
46 Train Loss 0.13461535 Test MSE 67.37851487523469 Test RE 0.11358314293898364
47 Train Loss 0.12480894 Test MSE 59.286382

28 Train Loss 5.598738 Test MSE 1085.4651678861237 Test RE 0.45589109645737363
29 Train Loss 5.430292 Test MSE 955.5600270417932 Test RE 0.42774225211000866
30 Train Loss 5.1832943 Test MSE 780.4955115044132 Test RE 0.3865790990749594
31 Train Loss 4.9928284 Test MSE 719.6081335363687 Test RE 0.371194212601364
32 Train Loss 4.6261077 Test MSE 670.9985255171288 Test RE 0.35843791928019497
33 Train Loss 4.5390496 Test MSE 713.8982941747188 Test RE 0.36971863146044665
34 Train Loss 4.4587264 Test MSE 717.8194694289788 Test RE 0.37073260382961437
35 Train Loss 4.326223 Test MSE 652.3596390588157 Test RE 0.3534245441886411
36 Train Loss 4.1411095 Test MSE 579.6099600789314 Test RE 0.3331356300994037
37 Train Loss 3.9633157 Test MSE 548.3846068863256 Test RE 0.32403788772633313
38 Train Loss 3.828769 Test MSE 501.20426432451484 Test RE 0.3097851128963345
39 Train Loss 3.6774795 Test MSE 430.60936419242574 Test RE 0.2871407957734458
40 Train Loss 3.4770453 Test MSE 376.652817531975 Test RE 0.

In [13]:
label

'1D_FODE_rowdy'

In [14]:
#3,4,8,9,13,14,18,19,23,24

In [15]:
import scipy.io as sio

In [16]:
for tune_reps in range(70):
#  if tune_reps not in s:
    label = "1D_FODE_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '1D_FODE_rowdy_tune0.mat'

In [ ]:
lrnr_tune[2]